# Building a RAG Application with IBM watsonx.ai and LangChain

## 1. Installing Dependencies

In [1]:
%pip install "protobuf==4.21.12"

%pip install -U "langchain>=0.3,<0.4" \
                 "langchain_ibm>=0.3,<0.4" \
                 "langchain_community>=0.3,<0.4" \
                 "langchain_chroma>=0.2,<0.3" \
                 sentence-transformers \
                 wget \
                 chromadb \
                 ibm-watsonx-ai

%pip install ibm-watson-machine-learning==1.0.312
%pip install "pandas==2.1.4"

## 2. Configuring IBM watsonx.ai Credentials

In [2]:
import os
import getpass
from ibm_watsonx_ai import Credentials

# The URL for the watsonx.ai API endpoint
url = 'https://us-south.ml.cloud.ibm.com'

# Enter the API key
api_key = getpass.getpass("Please enter your WML api key (hit enter): ")

# pass the credentials as an object
credentials = Credentials(
    api_key=api_key,
    url=url
)

# Getting the project ID
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

## 3. Preparing the Data

In [3]:
import wget

filename = 'state_of_the_union.txt'
url = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt'

if not os.path.isfile(filename):
    print("Downloading the data file...")
    wget.download(url, out=filename)
    print("Download complete.")
else:
    print("Data file already exists.")

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the document
loader = TextLoader(filename)
documents = loader.load()

# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(f"Document split into {len(texts)} chunks.")

## 4. Creating Embeddings and Vector Store

In [5]:
from ibm_watsonx_ai.foundation_models import Embeddings
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from langchain_chroma import Chroma

# Initializing the embedding model from watsonx.ai
embeddings = Embeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG,
    credentials=credentials,
    project_id=project_id
)

# Creating the Chroma vector store by embedding the text chunks
print("Creating vector store...")
docsearch = Chroma.from_documents(texts, embeddings)
print("Vector store created successfully.")

## 5. Setting up the LLM

In [6]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import WatsonxLLM

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 150, # Increased for potentially more detailed answers
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

watsonx_granite = WatsonxLLM(
    model_id="ibm/granite-13b-instruct-v2",
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

## 6. Building and Running the RAG Chain

In [7]:
from langchain.chains import RetrievalQA

# The 'stuff' chain_type is the simplest, as it "stuffs" all retrieved documents into the final prompt.
qa = RetrievalQA.from_chain_type(
    llm=watsonx_granite, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever()
)

## 7. Testing

In [8]:
query = "What did the president say about Ketanji Brown Jackson"
response = qa.invoke(query) # Using .invoke() which is the standard for new LangChain versions

print(f"Query: {response['query']}\n")
print("----------------------------------------------------------------\n")
print(f"Response: {response['result']}")

Query: What did the president say about Ketanji Brown Jackson
----------------------------------------------------------------
Response:  The president said that Ketanji Brown Jackson is one of our nation's top legal minds and that she will continue Justice Breyer's legacy of excellence. He also mentioned that she is a former top litigator in private practice and a former federal public defender.


In [ ]:
my_query = "What are the pillars of the Unity Agenda?" # Change this to your question
my_response = qa.invoke(my_query)

print(f"Query: {my_response['query']}\n")
print("----------------------------------------------------------------\n")
print(f"Response: {my_response['result']}")